# postgreSQL 연결

In [1]:
import psycopg2

In [2]:
conn_string = "host='localhost' dbname = 'seoul_dust' user= 'postgres' password='10co2pt' port = '5432'"
conn = psycopg2.connect(conn_string)
cur = conn.cursor()

In [9]:
# cur.execute("CREATE TABLE dust_20090102 (MSRDT real, MSRSTE_NIM text, NO2 real, O3 real, CO real, SO2 real, PM10 real, PM25 real);")

In [10]:
# conn.commit()

In [11]:
# cur.execute("COPY dust_20090102 FROM 'D:/DataScience/first-repository/raw_data/dust_20090102.csv' DELIMITER ',' CSV HEADER;")

In [12]:
# conn.commit()

# open API 데이터 수집

In [13]:
import requests
import pandas as pd
import os
import datetime as dt
import matplotlib.pyplot as plt
import urllib3

from time import sleep

In [14]:
def call_api(api_name, start_date, end_date, dir_name):

    # API 키는 공개하기 힘든 점 양해 바랍니다.

    api_key = open("./api_key/SEOUL_api_key.txt").readlines()[0].strip()

    url_format = 'http://openAPI.seoul.go.kr:8088/{api_key}/json/{api_name}/1/{end_index}/{date}'

    headers = {'content-type': 'application/json;charset=utf-8'}



    for date in pd.date_range(start_date, end_date).strftime("%Y%m%d"):

        # 최초 1회 Call은 해당 일자의 데이터 수를 확인한다.

        url = url_format.format(api_name=api_name, api_key=api_key, end_index=1, date=date)

        response = requests.get(url, headers=headers)

        end_index = response.json()[api_name]["list_total_count"]

        print("Max Count(%s): %s" % (date, end_index))



        # 해당 일자의 모든 데이터를 불러온다.

        url = url_format.format(api_name=api_name, api_key=api_key, end_index=end_index, date=date)

        response = requests.get(url, headers=headers)

        result = pd.DataFrame(response.json()[api_name]["row"])

        cur.execute("CREATE TABLE dust_%s (MSRDT real, MSRSTE_NIM text, NO2 real, O3 real, CO real, SO2 real, PM10 real, PM25 real);" % (date))
        conn.commit()
        
        result.to_csv("./raw_data/dust_%s.csv" % (date), index=False, encoding="UTF-8") #"EUC-KR")
        
        cur.execute("COPY dust_%s FROM 'D:/DataScience/first-repository/raw_data/dust_%s.csv' DELIMITER ',' CSV HEADER;" % (date, date))
        



        # API 부하 관리를 위해 0.5초 정도 쉬어 줍시다 (찡긋)

        sleep(0.5)

In [15]:
call_api("TimeAverageAirQuality", "2010-01-01", "2020-05-31", "dust")

Max Count(20100101): 600
Max Count(20100102): 600
Max Count(20100103): 600
Max Count(20100104): 600
Max Count(20100105): 600
Max Count(20100106): 600
Max Count(20100107): 600
Max Count(20100108): 600
Max Count(20100109): 600
Max Count(20100110): 600
Max Count(20100111): 600
Max Count(20100112): 600
Max Count(20100113): 600
Max Count(20100114): 600
Max Count(20100115): 600
Max Count(20100116): 600
Max Count(20100117): 600
Max Count(20100118): 600
Max Count(20100119): 600
Max Count(20100120): 600
Max Count(20100121): 600
Max Count(20100122): 600
Max Count(20100123): 598
Max Count(20100124): 600
Max Count(20100125): 600
Max Count(20100126): 600
Max Count(20100127): 600
Max Count(20100128): 600
Max Count(20100129): 600
Max Count(20100130): 600
Max Count(20100131): 600
Max Count(20100201): 600
Max Count(20100202): 600
Max Count(20100203): 600
Max Count(20100204): 600
Max Count(20100205): 598
Max Count(20100206): 600
Max Count(20100207): 600
Max Count(20100208): 600
Max Count(20100209): 600


In [26]:
for date_temp in ('20100202', '20200201'):
    cur.execute("COPY (SELECT * FROM dust_%s) TO 'D:/DataScience/first-repository/raw_data/dust_%s.csv' WITH CSV DELIMITER ',';" % (date_temp))

TypeError: not enough arguments for format string

In [32]:
conn.commit()
cur.execute("COPY (SELECT * FROM dust_20100203) TO 'D:/DataScience/first-repository/raw_data/dust_20100203.csv' WITH CSV DELIMITER ',';")